Image Helper

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# oRIGINAL CONTRIBUTOR spicyjpeg
import numpy, requests
from PIL import Image

def runHack():

    image = Image.open("texture.png")
    image.load()

    # NDR008" Easy swap to palette mode and resize :)
    image = image.convert('RGBA').convert('P', colors=3)
    image = image.resize((256,256))

    if image.mode != "P":
        raise RuntimeError("texture is not in indexed color format")

    ## Palette conversion

    # Analyze the palette's raw size to determine how many entries it contains. (If
    # there's a better/more efficient way to get the number of colors in a palette,
    # I don't know about it.)
    colorSize = { "RGB": 3, "RGBA": 4 }[image.palette.mode]
    palette   = image.palette.tobytes()
    numColors = len(palette) // colorSize

    # Extract the palette into a 2D array; Pillow makes this harder than it should
    # be. The array is then converted to 16bit to make sure there's enough room for
    # the large values temporarily produced by the 15bpp conversion (see below).
    paletteData = numpy \
        .frombuffer(palette, numpy.uint8) \
        .reshape(( numColors, colorSize )) \
        .astype(numpy.uint16)

    # For each RGB channel, convert it from 24bpp (0-255) to 15bpp (0-31) by
    # adjusting its levels and right-shifting it, then recombine the channels into
    # a single 16bpp 2D array which is going to be uploaded to the GPU as a 16xN
    # "image".
    # https://github.com/stenzek/duckstation/blob/master/src/core/gpu_types.h#L135
    red   = ((paletteData[:, 0] * 249) + 1014) >> 11
    green = ((paletteData[:, 1] * 249) + 1014) >> 11
    blue  = ((paletteData[:, 2] * 249) + 1014) >> 11

    paletteData = red | (green << 5) | (blue << 10)
    paletteData = paletteData.reshape(( 1, numColors ))

    ## Texture conversion

    # Get the image data as a 2D array of indices into the palette and ensure the
    # width of this array is even, as the GPU requires VRAM uploads to be done in
    # 16bit units (see upload()).
    imageData = numpy.asarray(image, numpy.uint8)

    if imageData.shape[1] % 2:
        padding   = numpy.zeros(imageData.shape[0], numpy.uint8)
        imageData = numpy.c_[ imageData, padding ]

    # If the texture is 4bpp, pack two pixels into each byte. This is done by
    # splitting the array into vertically interlaced odd/even columns and binary
    # OR-ing them after relocating the odd columns' values to the upper nibble. As
    # this operation halves the width of the image, another alignment check must be
    # performed afterwards.
    # https://numpy.org/doc/stable/user/basics.indexing.html#other-indexing-options
    if numColors <= 16:
        imageData = imageData[:, 0::2] | (imageData[:, 1::2] << 4)

        if imageData.shape[1] % 2:
            padding   = numpy.zeros(imageData.shape[0], numpy.uint8)
            imageData = numpy.c_[ imageData, padding ]

    ## Uploading

    def upload(x, y, numpyArray):
        requests.post(
            "http://localhost:8080/api/v1/gpu/vram/raw",
            data   = numpyArray.tobytes(),
            params = {
                "x":      str(x),
                "y":      str(y),
                # The width is always in 16bit units.
                "width":  str(numpyArray.shape[1] // 2 * numpyArray.itemsize),
                "height": str(numpyArray.shape[0])
            }
        )
    
    AllFlag = True
    if AllFlag:

        imageData2 = imageData
        for i in range(448, 800, 128):
            upload(i, 0, imageData2)
            upload(i, 256, imageData2)
        for i in range(640, 1024-127, 128):
            upload(i, 0, imageData)
            upload(i, 256, imageData)
    else:

        for i in range(640, 1024-127, 128):
            upload(i, 0, imageData)
            upload(i, 256, imageData)


In [1]:
# just to find screen limits
screenXY = []

import pyautogui as py
import time
for i in range(0, 2):
    print("position", i+1)
    time.sleep(3)
    screenXY.append(py.position())
x1,y1 = screenXY[0]
x2,y2 = screenXY[1]
print(x1,y1,x2,y2)

position 1
position 2
81 136 598 525


In [2]:
x1=81
y1=136
x2=598 
y2=525

In [3]:
import numpy as np
import d3dshot
import cv2
from time import time


def screen_record_d3dshot_play():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    start_time = time()
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.GaussianBlur(printscreen, (5, 5), 0)
        canny = cv2.Canny(grey, threshold1=50, threshold2=100)
        enhanced = cv2.equalizeHist(canny)
        cv2.imshow('window', cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    end_time = time()
    cv2.destroyAllWindows()
    screen_buffer.stop()
    return (end_time - start_time)


In [5]:
screen_record_d3dshot_play()

Only 1 instance of D3DShot is allowed per process! Returning the existing instance...


26.374245643615723

In [2]:
x1=25
y1=84
x2=565 
y2=486

In [6]:
import numpy as np
import d3dshot
import cv2


def gray_equal_blur_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        enhanced = cv2.equalizeHist(grey)
        blur = cv2.GaussianBlur(enhanced, (5, 5), 0)
        canny = cv2.Canny(blur, threshold1=50, threshold2=100)
        cv2.imshow('window', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


def gray_blur_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (5, 5), 0)
        canny = cv2.Canny(blur, threshold1=50, threshold2=100)
        cv2.imshow('window', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


def gray_equ_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        enhanced = cv2.equalizeHist(grey)
        canny = cv2.Canny(enhanced, threshold1=50, threshold2=100)
        cv2.imshow('window', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


def gray_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(grey, threshold1=50, threshold2=100)
        cv2.imshow('window', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()
    
    
def gray_blur_equ_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (3, 3), 0)
        enhanced = cv2.equalizeHist(blur)
        canny = cv2.Canny(enhanced, threshold1=50, threshold2=100)
        cv2.imshow('window', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


In [29]:
print("gray_equal_blur_canny()")
gray_blur_canny()

print("gray_equ_canny()")
gray_equ_canny()

print("gray_canny()")
gray_canny()

print("gray_blur_equ_canny")
gray_blur_equ_canny()


gray_equal_blur_canny()
Only 1 instance of D3DShot is allowed per process! Returning the existing instance...
gray_equ_canny()
Only 1 instance of D3DShot is allowed per process! Returning the existing instance...
gray_canny()
Only 1 instance of D3DShot is allowed per process! Returning the existing instance...
gray_blur_equ_canny
Only 1 instance of D3DShot is allowed per process! Returning the existing instance...


In [3]:
import numpy as np
import d3dshot
import cv2

def gray_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(grey, threshold1=50, threshold2=100)
        cv2.imshow('grey', cv2.cvtColor(grey, cv2.COLOR_BGR2RGB))
        cv2.imshow('canny', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()
    
gray_canny()


Only 1 instance of D3DShot is allowed per process! Returning the existing instance...


In [2]:
import numpy as np
import d3dshot
import cv2


def trial_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        hsv = cv2.cvtColor(printscreen, cv2.COLOR_BGR2HSV)
        
        # Threshold of blue in HSV space
        lower_blue = np.array([60, 35, 140])
        upper_blue = np.array([180, 255, 255])
        mask = cv2.inRange(hsv, lower_blue, upper_blue)
        
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(grey, threshold1=50, threshold2=100)
        
        result = cv2.bitwise_and(printscreen, printscreen, mask=mask)
        
        cv2.imshow('grey', cv2.cvtColor(grey, cv2.COLOR_BGR2RGB))
        cv2.imshow('canny', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        cv2.imshow('mask', cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
        cv2.imshow('blue', cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


trial_canny()



GT2 Optimum Tahiti

In [7]:
import numpy as np
import d3dshot
import cv2


def gray_canny():
    screen_buffer = d3dshot.create(
        capture_output="numpy")
    screen_buffer.display = screen_buffer.displays[0]
    while (True):
        printscreen = screen_buffer.screenshot(region=(x1, y1, x2, y2))
        grey = cv2.cvtColor(printscreen, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(grey, threshold1=150, threshold2=200)
        cv2.imshow('grey', cv2.cvtColor(grey, cv2.COLOR_BGR2RGB))
        cv2.imshow('canny', cv2.cvtColor(canny, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
    screen_buffer.stop()


gray_canny()


Only 1 instance of D3DShot is allowed per process! Returning the existing instance...


Not working yet, but redux has an option to overwrite vram, so it could be possible to substitute textures

In [9]:
import requests
import numpy as np
from PIL import Image, ImageOps

image = Image.open("crap.bmp")
image = image.convert("P")
pix_val = list(image.getdata())
response = requests.post(
    'http://localhost:8080/api/v1/gpu/vram/raw?x=420&y=150&width=420&height=420', files=dict(pixels=pix_val))


ValueError: too many values to unpack (expected 4)